<a href="https://colab.research.google.com/github/iljf/NLU_project_team1/blob/main/KLUE_ROBERTA_base_iljf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KLUE - ROBERTA-BASE

klue/roberta-base 모델을 KLUE 내 STS 데이터셋을 활용하여 모델을 훈련

*   source code : https://github.com/UKPLab/sentence-transformers




In [ ]:
!pip install transformers sentence-transformers datasets wandb 

In [73]:
import pandas as pd
import os
import sys
import re
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
import math
import logging
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sklearn.metrics import f1_score
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CosineSimilarity, MSELoss
from torch.nn.utils import clip_grad_norm_

In [3]:
#logger 초기화
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [4]:
#argument setting
task = "sts"
model_checkpoint = "klue/roberta-base"

In [5]:
from datasets import load_dataset
dataset = load_dataset('klue', task)

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [7]:
dataset.keys()

dict_keys(['train', 'validation'])

In [8]:
dataset.shape

{'train': (11668, 5), 'validation': (519, 5)}

In [9]:
dataset['train']

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 11668
})

In [10]:
dataset['validation']

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 519
})

In [11]:
!git clone https://github.com/KLUE-benchmark/KLUE.git

Cloning into 'KLUE'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 157 (delta 18), reused 9 (delta 9), pack-reused 133
Receiving objects: 100% (157/157), 49.11 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Checking out files: 100% (29/29), done.


In [12]:
KLUE_dataset = pd.read_json('/content/KLUE/klue_benchmark/klue-sts-v1.1/klue-sts-v1.1_train.json')

In [13]:
KLUE_dataset.shape

(11668, 6)

In [14]:
KLUE_dataset.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [15]:
KLUE_dataset.duplicated(['sentence1', 'sentence2']).sum()

7

In [16]:
KLUE_dataset = KLUE_dataset.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [17]:
labels = KLUE_dataset.labels.to_list()
labels = pd.DataFrame(labels)
print(len(labels))
labels

11661


,label,real-label,binary-label
0,3.7,3.714286,1
1,0.0,0.000000,0
2,0.3,0.333333,0
3,0.6,0.571429,0
4,4.7,4.714286,1
...,...,...,...
11656,4.0,4.000000,1
11657,0.0,0.000000,0
11658,3.7,3.666667,1
11659,4.7,4.714286,1


In [18]:
KLUE_dataset = KLUE_dataset[['sentence1', 'sentence2']].join(labels[['real-label']])
KLUE_dataset.head()

,sentence1,sentence2,real-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.714286
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.333333
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.571429
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.714286


In [19]:
!git clone https://github.com/kakaobrain/KorNLUDatasets/

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [20]:
!git clone https://github.com/warnikchow/paraKQC

Cloning into 'paraKQC'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 48 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [21]:
dir = '/content/KorNLUDatasets/KorSTS/sts-train.tsv'

In [22]:
Korsts_dataset = pd.read_csv(dir, delimiter="\t+")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [23]:
Korsts_dataset.shape

(5749, 7)

In [24]:
Korsts_dataset.head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.


In [25]:
Korsts_dataset = Korsts_dataset.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [26]:
Korsts_dataset.isnull().sum()

genre        0
filename     0
year         0
id           0
score        0
sentence1    0
sentence2    0
dtype: int64

In [27]:
Korsts_dataset.rename(columns = {'score':'real-label'},inplace=True)

In [28]:
Korsts_dataset = Korsts_dataset[['sentence1', 'sentence2', 'real-label']]
Korsts_dataset.head()

,sentence1,sentence2,real-label
0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,5.00
1,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,3.80
2,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,3.80
3,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.,2.60
4,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.,4.25


In [29]:
dir2 = '/content/paraKQC/data/paraKQC_v1.txt'

In [30]:
ParaKQC_raw = pd.read_csv(dir2, sep="\t")

In [31]:
ParaKQC_raw.head()

,0,0.1,메일을 다 비울까 아니면 안읽은 것만 지울까?
0,0,0,메일 중에 안읽은 것만 지울까? 다 지울까?
1,0,0,안읽은 메일만 지워 다지워?
2,0,0,다 지울까 안읽은 메일만 지울까?
3,0,0,전체를 비울까 안읽은 것만 비울까?
4,0,0,안읽은 메일만 지울꺼야? 아니면 다 지울꺼야?


In [32]:
df = pd.concat([KLUE_dataset,Korsts_dataset],axis=0, join='inner', ignore_index=True)

In [33]:
import html
import regex as re
from bs4 import BeautifulSoup

def preprocess(sentence):
    sen = BeautifulSoup(html.unescape(sentence), 'html.parser').text     # html parse
    sen = sen.replace("\n", " ")                                    # \n
    sen = re.sub('"',' ', sen)                                      # 따옴표 
    sen = re.sub("[^a-zA-Z0-9가-힣]", " ", sen)                  #영문, 한글, 숫자 만
    sen = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', sen)    
    return sen



def preprocess_train(df):
    sentence1 = df['sentence1'].tolist()
    sentence2 = df['sentence2'].tolist()
    real_label = df['real-label'].tolist()

    processed1 = []
    processed2 = []

    for sen1 in sentence1:
        processed1.append(preprocess(sen1))
    for sen2 in sentence2:
        processed2.append(preprocess(sen2))
    
    processed_df = pd.DataFrame(list(zip(processed1, processed2, real_label)),
                        columns = ['sentence1', 'sentence2', 'real-label'])


    return processed_df

In [34]:
df = preprocess_train(df)

In [35]:
df.shape

(17361, 3)

In [36]:
df.head()

,sentence1,sentence2,real-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다,3.714286
1,위반행위 조사 등을 거부 방해 기피한 자는 500만원 이하 과태료 부과 대상이다,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.333333
3,긴급 고용안정지원금은 지역고용대응 등 특별지원금 지자체별 소상공인 지원사업 취업...,고용보험이 1차 고용안전망이라면 국민취업지원제도는 2차 고용안전망입니다,0.571429
4,호스트의 답장이 늦으나 개선될 것으로 보입니다,호스트 응답이 늦었지만 개선될 것으로 보입니다,4.714286


In [37]:
df.tail()

,sentence1,sentence2,real-label
17356,폭풍우 클로다흐가 영국을 강타하면서 심한 강풍,메르켈은 나토와 라트비아의 연대를 약속한다,0.0
17357,리비아 테러리스트들이 공습에 대한 복수로 찍은 수십 명의 이집트 인질들,나일강에서 더 많은 시체가 발견되면서 이집트 보트 충돌 사고 사망자 수가 증가한다,0.0
17358,바레인으로 향하는 대통령,시 주석 에볼라 퇴치를 계속 돕기 위한 중국,0.0
17359,중국 인도는 양국 관계를 증진시키겠다고 맹세한다,중국은 불안한 주식 거래자들을 안심시키기 위해 뒤뚱거리고 있다,0.0
17360,푸틴 대변인 도핑 혐의는 근거 없는 것으로 보인다,가장 최근의 심한 날씨 토네이도 후 텍사스에서 1명 사망,0.0


In [38]:
test = pd.read_json('/content/KLUE/klue_benchmark/klue-sts-v1.1/klue-sts-v1.1_dev.json')

In [39]:
test = test.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [40]:
labels = test.labels.to_list()
labels = pd.DataFrame(labels)
print(len(labels))
labels

519


,label,real-label,binary-label
0,4.9,4.857143,1
1,1.4,1.428571,0
2,1.3,1.285714,0
3,3.7,3.714286,1
4,2.5,2.500000,0
...,...,...,...
514,2.2,2.200000,0
515,2.8,2.833333,0
516,0.3,0.333333,0
517,0.3,0.333333,0


In [41]:
test = test[['sentence1', 'sentence2']].join(labels[['real-label']])
test.head()

,sentence1,sentence2,real-label
0,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",4.857143
1,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,1.428571
2,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,1.285714
3,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",3.714286
4,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,2.500000


In [42]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModel
from transformers import get_linear_schedule_with_warmup, get_constant_schedule


In [44]:
model_checkpoint

'klue/roberta-base'

In [45]:
tokenizer = AutoTokenizer.from_pretrained("rurupang/roberta-base-finetuned-sts")

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/735k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [46]:
class CustomDataset(Dataset):
    def __init__(self, sentence1, sentence2, real_label):
        self.X1 = sentence1 #list str
        self.X2 = sentence2 #list str
        self.Y = real_label #list float

    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, index):
        return self.X1[index], self.X2[index], self.Y[index]

In [47]:
def generate_dataset(df, flag):
    sen_one = df['sentence1'].tolist()
    sen_two = df['sentence2'].tolist()
    real_lab = df['real-label'].tolist()
    
    if flag:
        return CustomDataset(sen_one, sen_two, real_lab)
    else:
        return CustomDataset(sen_one, sen_two, lab)

In [48]:
train, val = train_test_split(df, test_size=0.1, shuffle=True)

In [49]:
train_dataset = generate_dataset(train, True)
val_dataset = generate_dataset(val, True)
test_dataset = generate_dataset(test, True)

In [50]:
# seed 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [51]:
max_length = 512

In [52]:
def CustomCollateFn(batch):
    sen_one_list = []
    sen_two_list = []
    label_list = []


    for sen_one, sen_two, label in batch:
        sen_one_list.append(sen_one)
        sen_two_list.append(sen_two)
        label_list.append(label/5.0)
    
    tokenized_sen_one = tokenizer(sen_one_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=max_length, return_tensors='pt')
    tokenized_sen_two = tokenizer(sen_two_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=max_length, return_tensors='pt')

    label_list = torch.Tensor(label_list)


    return (tokenized_sen_one, tokenized_sen_two, label_list)

def CustomCollateFn_dev(batch):
    sen_one_list = []
    sen_two_list = []
    label_list = []


    for sen_one, sen_two, label in batch:
        sen_one_list.append(sen_one)
        sen_two_list.append(sen_two)
        label_list.append(label)

    tokenized_sen_one = tokenizer(sen_one_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=max_length, return_tensors='pt')
    tokenized_sen_two = tokenizer(sen_two_list, add_special_tokens=True, padding='max_length',
                                truncation=True, max_length=max_length, return_tensors='pt')

    label_list = torch.Tensor(label_list)

    return (tokenized_sen_one, tokenized_sen_two, label_list)

In [53]:
def mean_pooling_fn(output, attention_mask):
    embedding = output.last_hidden_state # (batch len, longest sentence length, 1024)
    att_msk = attention_mask # (batch_len, 1024)
    mask = att_msk.unsqueeze(-1).expand(output.last_hidden_state.size()).float() # (batch len, longest sentence length, 1024)
    masked_embedding = output.last_hidden_state * mask # (batch_len, longest sen len, 1024)
    me_sum = torch.sum(masked_embedding, 1) # (batch_len, 1024)
    ms_sum = torch.clamp(mask.sum(1), min=1e-9) # (batch_len, 1024)
    mean_pool = me_sum/ms_sum # batch_len, 1024
    return mean_pool

In [54]:
def mean_pooling_fn(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-8)

In [55]:
class CustomPooling(nn.Module):
    def __init__(self):
        super(CustomPooling, self).__init__()

        self.robert = AutoModel.from_pretrained("klue/roberta-base")


        self.cos_score = nn.Sequential(
            nn.Identity()
        )
    
    def forward(self, senone, sentwo):
        output_one = self.robert(input_ids=senone['input_ids'], attention_mask=senone['attention_mask'],
                             token_type_ids=senone['token_type_ids'])
        output_two = self.robert(input_ids=sentwo['input_ids'], attention_mask=sentwo['attention_mask'],
                             token_type_ids=sentwo['token_type_ids'])

        pooled_one = mean_pooling_fn(output_one, senone['attention_mask'])
        pooled_two = mean_pooling_fn(output_two, sentwo['attention_mask'])


        cos_sim = torch.cosine_similarity(pooled_one, pooled_two)
        logit = self.cos_score(cos_sim)

        return logit

In [56]:
def initializer(input_dataloader, epochs):

    model = CustomPooling()
    optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
    print(f'total step: {len(input_dataloader) * epochs}')

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps = round(len(input_dataloader)*0.1),
        num_training_steps = len(input_dataloader) * epochs,

    )

    return model, optimizer, scheduler

In [75]:
def initializer(input_dataloader, epochs):
    """
    설정에 맞춰서 wandb sweep 실행.
    """
    wandb.init(config=sweep_config)
    model = CustomPooling()   
    w_config = wandb.config   

    optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8) 
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps = round(len(input_dataloader)*0.1),
        num_training_steps = len(input_dataloader) * epochs,

    )
    print(f'total step: {len(input_dataloader) * epochs}') 
    text_table = wandb.Table(columns=["epoch", "step", "text", 'true_label', 'pred_label'])
    wandb.log({f"error-text-{wandb.run.name}" : text_table})
    return model, optimizer, scheduler

In [57]:
metric_name = "pearsonr"

In [58]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    
    # Optimizer
    learning_rate=1e-5, # 5e-5
    weight_decay=0.01,  # 0
    # warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_' + metric_name,
    evaluation_strategy = "epoch",


    # Logging
    logging_dir=logging_dir,
    report_to='wandb',
)

In [59]:
def train(model, loss_fct, scheduler, optimizer, train_dataloader, valid_dataloader, epochs):
    for epoch in range(epochs):
        print(f'****** STARTING TO TRAIN EPOCH #{epoch} ******')

        wandb.watch(model, log="all", log_freq = 10)
        total_loss = 0
        batch_loss = 0
        batch_count = 0

        model.train()
        model.to(device)

        for step, batch in enumerate(train_dataloader):
            batch_count += 1
            batch = tuple(items.to(device) for items in batch)

            (x_batch_one, x_batch_two, y_batch) = batch

            model.zero_grad()

            logit = model(x_batch_one, x_batch_two)
            loss = loss_fct(logit, y_batch)

            batch_loss += loss.item()
            total_loss += loss.item()

            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            if(step % 10 == 0 and step != 0):
                wandb.log({'train_loss': batch_loss / batch_count, 'train_lr': optimizer.param_groups[0]['lr']})
                print(f"Step : {step + 1}, train Loss : {batch_loss / batch_count:.4f}")                      
                # reset 
                batch_loss, batch_count = 0,0

        wandb.log({'total_train_loss': total_loss / (step + 1), 'total_train_lr': optimizer.param_groups[0]['lr'], "epoch" : (epoch + 1)})
  

        print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
        print(f"*****Epoch {epoch} Train Finish*****\n")
        save_checkpoint(".", model, optimizer, scheduler, epoch, total_loss/(step+1))
            torch.cuda.empty_cache()
       
        if valid_dataloader is not None:
            print(f"*****Epoch {epoch} Valid Start*****")
            valid_loss, valid_pearson, valid_f1 = validate(model, loss_fct, valid_dataloader)
            print(f"Epoch {epoch} Valid Loss : {valid_loss} Valid Pearsonr : {valid_pearson} ValidF1 : {valid_f1}")
            print(f"*****Epoch {epoch} Valid Finish*****\n")
  

    print('** Train Completed! **')

In [60]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, cer, chrf, code_eval, comet, competition_math, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, matthews_correlation, mauve, mean_iou, meteor, mse, pearsonr, perplexity, precision, recall, roc_auc, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, ter, wer, wiki_split, xnli, xtreme_s


In [61]:
metric_name = "pearsonr"
metric_pearsonr = load_metric(metric_name)
metric_name2 = "f1"
metric_f1 = load_metric(metric_name2)

In [62]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    pr = metric_pearsonr.compute(predictions=predictions,
                                  references=labels)
    return pr

    f1 = metric_f1.compute(predictions=predictions,
                                  references=labels)
    return {'pearsonr' : pr,
             'f1' : f1}

In [ ]:
!pip install audtorch

In [64]:
def validate(model, loss_fct, valid_dataloader):

    model.eval()
    model.to(device)

    total_loss = 0
    total_acc = 0
    all_prediction = []
    all_reallabel = []

    for step, batch in enumerate(valid_dataloader):
        batch = tuple(items.to(device) for items in batch)

        (x_batch_one, x_batch_two, batch_y) = batch

        with torch.no_grad():
            logit = model(x_batch_one, x_batch_two)

        logit = logit*5
        loss = loss_fct(logit, batch_y)
        total_loss += loss.item()


        logit = logit[:].cpu()
        batch_y = batch_y.cpu()

        print(f'Step: {step},  Pearson: {pearsonr(logit, batch_y)}')

        all_prediction = all_prediction + logit.tolist()
        all_reallabel = all_reallabel + batch_y.tolist()

    #pearson

    pred = torch.Tensor(all_prediction) # x
    real = torch.Tensor(all_reallabel) # y
    
    pearson = pearsonr(pred, real)
    
    #loss
    total_loss = total_loss / (step+1)

    #f1
    fone = f1_process(pred, real)

    wandb.log({'total_valid_loss': total_loss, "total_f1_score ": fone, "total_pearsonr" : pearson})  
    print('total_valid_loss : ', total_loss, "total_f1_score : ",  fone,  "total_pearsonr :", pearson)  
    return total_loss, pearson, fone

In [65]:
def f1_process(pred, real):
    bin_real = []
    bin_pred = []

    for index in range(len(real)):
        if real[index] < 3:
            bin_real.append(0)
        else:
            bin_real.append(1)
    
        if pred[index] < 3:
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    return f1_score(bin_real, bin_pred)

In [66]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size = 32,
    sampler = RandomSampler(train_dataset),
    collate_fn = CustomCollateFn,
)
valid_dataloader = DataLoader(
    val_dataset,
    batch_size = 8,
    sampler = SequentialSampler(val_dataset),
    collate_fn = CustomCollateFn_dev,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size = 8,
    sampler = SequentialSampler(test_dataset),
    collate_fn = CustomCollateFn_dev,
)

In [67]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [69]:
id = wandb.util.generate_id()
print(id)

2pedtbv7


In [70]:
sweep_config = {
    
    "name" : "sts_r1",   
    "method": "bayes",
    "metric": {
        "name" : "total_pearsonr", 
        "goal" : "maximize"
                },
    
    "parameters": { 
        "epochs" : {
            "distribution" : "categorical",
            "values" : [4]},
        "learning_rate" : {
            "distribution" : "categorical",
            "values" : [1e-5, 2e-5, 5e-5]},                     
        "eps" : {
            "distribution" : "categorical",
            "values" : [1e-8]
        },
        "train_batch_size" : {
            "distribution" : "categorical",
            "values" : [32]
        },
        "valid_batch_size" : {
            "distribution" : "categorical",
            "values" : [8]
        },
        "warm_up_ratio" : {
            "distribution" : "categorical",
            "values" : [0, 0.1]  #[0, 0.1, 0.2]
        },
    },         
    "early_terminate" : {
        "type": "hyperband", # metric이 2번 이상 개선되지 않을 경우 조기 종료
        "min_iter" : 2,
        "eta" : 2
        }
}

In [71]:
def run_sweeep(config=None):
    """
    설정에 맞춰서 wandb sweep 실행.
    """
    model = CustomPooling() 
    wandb.init(config=config)
    w_config = wandb.config   
    
    optimizer = AdamW(model.parameters(), lr = w_config.learning_rate, eps =  w_config.eps) 
    num_training_steps = w_config.epochs * len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps= (num_training_steps * w_config.warm_up_ratio),
                                                num_training_steps = num_training_steps)
    loss_fct = MSELoss()
    train(model, loss_fct, scheduler, optimizer, train_dataloader, valid_dataloader, w_config.epochs)

In [76]:
sweep_id = wandb.sweep(sweep_config, project = "sts_r1")
wandb.agent(sweep_id, run_sweeep, count = 4)

Create sweep with ID: d7zi6n3j
Sweep URL: https://wandb.ai/iljf/sts_r1/sweeps/d7zi6n3j


wandb: Agent Starting Run: p71cduw1 with config:
wandb: 	epochs: 4
wandb: 	eps: 1e-08
wandb: 	learning_rate: 5e-05
wandb: 	train_batch_size: 32
wandb: 	valid_batch_size: 8
wandb: 	warm_up_ratio: 0
Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkp

****** STARTING TO TRAIN EPOCH #0 ******


Run p71cduw1 errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run p71cduw1 errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: 4owpqv4c with config:
wandb: 	epochs: 4
wandb: 	eps: 1e-08
wandb: 	learning_rate: 1e-05
wandb: 	train_batch_size: 32
wandb: 	valid_batch_size: 8
wandb: 	warm_up_ratio: 0
Some weights of the model checkpoint at klue/rober

****** STARTING TO TRAIN EPOCH #0 ******


Run 4owpqv4c errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run 4owpqv4c errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: 3g4givex with config:
wandb: 	epochs: 4
wandb: 	eps: 1e-08
wandb: 	learning_rate: 2e-05
wandb: 	train_batch_size: 32
wandb: 	valid_batch_size: 8
wandb: 	warm_up_ratio: 0.1
Some weights of the model checkpoint at klue/rob

****** STARTING TO TRAIN EPOCH #0 ******


Run 3g4givex errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run 3g4givex errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.78 GiB already allocated; 19.75 MiB free; 14.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
